### Analysis software development v2 

Goal: establish smooth reading of the binary DAQ datafiles

In [1]:
import sys
sys.path.insert(0,'python/')
import struct
import matplotlib.pyplot as plt
from scipy.stats import norm


# import data handling class
from DT5550 import *


Open a datafile and read the data

In [ ]:
#
# open the datfile / data directory
#
    
#
# loop over all the events
#
dt_max = 10

Etot = []
tt = []
pp = []
for i in range(8):
    tt.append([])
    pp.append([])
tdif = []

nmax = 9999

#    while io.read_event() == 0:
#        #
#        # data: io.Q[0:ndet-1] -> energy
#        #       io.ph[0:ndet-1] -> peak height
#        #       io.t[0:ndet-1] -> time without timewalk correction
#        #       io.tc[0:ndet-1] -> time with timewalk correction
#        #       io.valid[0:ndet-1] -> valid bit = 1 for valid data
#        #       io.R[0:ndet-1] -> peak/area ratio


# tagged
#io = DT5550(indir='C:/data/20220118_100036')

io = DT5550(indir='C:/data/20220114_094646/')

nf = 0

dtime = []
for file in io.filenames:
    io.open_data(file)
    nf = nf + 1
    if nf>nmax:
        break
    while io.read_event() == 0: 
        # data: io.Q[0:ndet-1] -> energy
        #       io.ph[0:ndet-1] -> peak height
        #       io.t[0:ndet-1] -> time without timewalk correction
        #       io.tc[0:ndet-1] -> time with timewalk correction
        #       io.valid[0:ndet-1] -> valid bit = 1 for valid data
        #       io.R[0:ndet-1] -> peak/area ratio
        nh = io.valid.sum()
        
        Etot.append(io.Qraw.sum())
#         for idet in range(N_DETECTOR):
#             if io.valid[idet] == 1:
#                 tt[idet].append(io.tc[idet])
                
        for idet in range(8):
            if io.valid[idet] == 1 and io.tc[idet]<100:
                tt[idet].append(io.tc[idet])
                pp[idet].append(io.ph[idet])

        
        tdif = []
        if nh == 2:
            
            for idet in range(N_DETECTOR):
                if io.valid[idet] == 1:
                    tdif.append(idet)
                    
            peak1 = tdif[0]
            peak2 = tdif[1]  
            dtime.append(io.tc[peak2] - io.tc[peak1])
            
#         print(io.ph[6])    
#         print(io.timewalk_correct(6))
print('Processing done')

Etot = np.array(Etot,dtype=object)
tt = np.array(tt,dtype=object)
pp = np.array(pp,dtype=object)


#r = np.array(r, dtype=object)

DT5550:: Data recorded with config:  C:/data/20220114_094646\config_20220114_094646.json
DT5550:: Open data file: C:/data/20220114_094646\data_20220114_094646_0.raw


python\DT5550.py:17: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "word_unpack" failed type inference due to: No implementation of function Function(<built-in function frombuffer>) found for signature:
 
 >>> frombuffer(readonly bytes(uint8, 1d, C), dtype=class(uint8), offset=int64)
 
There are 4 candidate implementations:
  - Of which 4 did not match due to:
  Overload in function '_OverloadWrapper._build.<locals>.ol_generated': File: numba\core\overload_glue.py: Line 131.
    With argument(s): '(readonly bytes(uint8, 1d, C), dtype=class(uint8), offset=int64)':
   Rejected as the implementation raised a specific error:
     TypingError: Failed in nopython mode pipeline (step: nopython frontend)
   No implementation of function Function(<intrinsic stub>) found for signature:
    
    >>> stub(readonly bytes(uint8, 1d, C), class(uint8), int64)
    
   There are 2 candidate implementations:
     - Of which 2 did not match due to:
   

DT5550:: Open data file: C:/data/20220114_094646\data_20220114_094646_1.raw
DT5550:: Open data file: C:/data/20220114_094646\data_20220114_094646_2.raw
DT5550:: Open data file: C:/data/20220114_094646\data_20220114_094646_3.raw
DT5550:: Open data file: C:/data/20220114_094646\data_20220114_094646_4.raw
DT5550:: Open data file: C:/data/20220114_094646\data_20220114_094646_5.raw
DT5550:: Open data file: C:/data/20220114_094646\data_20220114_094646_6.raw
DT5550:: Open data file: C:/data/20220114_094646\data_20220114_094646_7.raw
DT5550:: Open data file: C:/data/20220114_094646\data_20220114_094646_8.raw
DT5550:: Open data file: C:/data/20220114_094646\data_20220114_094646_9.raw
DT5550:: Open data file: C:/data/20220114_094646\data_20220114_094646_11.raw
DT5550:: Open data file: C:/data/20220114_094646\data_20220114_094646_12.raw
DT5550:: Open data file: C:/data/20220114_094646\data_20220114_094646_13.raw
DT5550:: Open data file: C:/data/20220114_094646\data_20220114_094646_14.raw
DT5550::

#### Define fit functions

In [ ]:
h = plt.hist(Etot,bins=100,range=(0,3000))
plt.yscale('log')


print(len(Etot))

In [ ]:
plt.figure(figsize=(12,18))
for idet in range(N_DETECTOR):
    plt.subplot(4, 2, 1 + idet)
    h = plt.hist(tt[idet],bins=100,range=(-100,500))
    print(idet, 'n = ', len(tt[idet]))
#     if idet == 7:
#        print(idet,tt[idet])

In [ ]:
h = plt.hist(dtime,bins=100,range=(-100,100))

In [ ]:
75/1800*2**14


In [ ]:
plt.figure(figsize=(12,18))
for idet in range(N_DETECTOR):
    plt.subplot(4, 2, 1 + idet)
    h = plt.hist(pp[idet],bins=200,range=(0,10000))
    print(idet, 'n = ', len(tt[idet]))
#     if idet == 7:
#        print(idet,tt[idet])